<a href="https://colab.research.google.com/github/TashreefMuhammad/CSE4238_Assignment2/blob/main/170104014_exp_02_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSE 4238 | Assignment 2
---

## Prepare Dataset
---
Download and unzip the dataset for experiment.

As my ID is even, [Dataset A](https://drive.google.com/file/d/1txyKhs1Zt5AKswGGK9VI_jE0JNHuQT85/view) is being downloaded as per instruction

In [ ]:
# The dataset => https://drive.google.com/file/d/1txyKhs1Zt5AKswGGK9VI_jE0JNHuQT85/view
!gdown --id 1txyKhs1Zt5AKswGGK9VI_jE0JNHuQT85

# Unzip the dataset
!unzip "/content/Dataset A.zip" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/training-a/a14702.png  
  inflating: /content/training-a/a14703.png  
  inflating: /content/training-a/a14704.png  
  inflating: /content/training-a/a14705.png  
  inflating: /content/training-a/a14706.png  
  inflating: /content/training-a/a14707.png  
  inflating: /content/training-a/a14708.png  
  inflating: /content/training-a/a14709.png  
  inflating: /content/training-a/a14710.png  
  inflating: /content/training-a/a14711.png  
  inflating: /content/training-a/a14712.png  
  inflating: /content/training-a/a14713.png  
  inflating: /content/training-a/a14714.png  
  inflating: /content/training-a/a14715.png  
  inflating: /content/training-a/a14716.png  
  inflating: /content/training-a/a14717.png  
  inflating: /content/training-a/a14718.png  
  inflating: /content/training-a/a14719.png  
  inflating: /content/training-a/a14720.png  
  inflating: /content/training-a/a14721.png  
  inflating: /content/trainin

## Import Required Libraries
---
Importing libraries that will be required for the experiment. As per instruction, PyTorch will be used in this experiment

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

from PIL import Image

## Read the Original Data
---
Read the original data from the CSV file found

In [ ]:
data = pd.read_csv('/content/training-a.csv')

data

,filename,original filename,scanid,digit,database name original,contributing team,database name
0,a00000.png,Scan_58_digit_5_num_8.png,58,5,BHDDB,Buet_Broncos,training-a
1,a00001.png,Scan_73_digit_3_num_5.png,73,3,BHDDB,Buet_Broncos,training-a
2,a00002.png,Scan_18_digit_1_num_3.png,18,1,BHDDB,Buet_Broncos,training-a
3,a00003.png,Scan_166_digit_7_num_3.png,166,7,BHDDB,Buet_Broncos,training-a
4,a00004.png,Scan_108_digit_0_num_1.png,108,0,BHDDB,Buet_Broncos,training-a
...,...,...,...,...,...,...,...
19697,a19697.png,Scan_81_digit_4_num_3.png,81,4,BHDDB,Buet_Broncos,training-a
19698,a19698.png,Scan_70_digit_3_num_6.png,70,3,BHDDB,Buet_Broncos,training-a
19699,a19699.png,Scan_253_digit_8_num_4.png,253,8,BHDDB,Buet_Broncos,training-a
19700,a19700.png,Scan_27_digit_3_num_6.png,27,3,BHDDB,Buet_Broncos,training-a


## Remove Columns from Dataset
---

Remove columns from the dataset as per requirement

In [ ]:
columns = data.columns.to_list()
# print(columns)

columns.remove('filename')
columns.remove('digit')
# print(columns)

data.drop(labels = columns, axis = 1, inplace = True)

# Shuffle Dataset
data = data.sample(frac = 1, random_state = 14).reset_index(drop = True)
data

,filename,digit
0,a18483.png,0
1,a07473.png,5
2,a18214.png,6
3,a06037.png,2
4,a12149.png,8
...,...,...
19697,a12025.png,5
19698,a18838.png,7
19699,a09484.png,2
19700,a13656.png,4


## Declaring Hyperparameters
---

Declaring the values of some Hyperparameters

In [ ]:
no_hidden_layer = 3
no_nodes = 200
iterations = 20000
learning_rate = 0.01
batch_size = 256
input_dim = 180 * 180

split_val = int(0.8 * data.shape[0])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Preparing Data
---
Preparing the data for training using PyTorch

In [ ]:
class Dataset(Dataset):
  def __init__(self, data, dir, transform = None):
    self.data = data
    self.dir = dir
    self.transform = transform
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    item = self.data.iloc[index]
    
    path = self.dir + "/" + item[0]
    image = Image.open(path).convert('L')
    label = item[1]
    
    if self.transform is not None:
      image = self.transform(image)
      
    return image, label

In [ ]:
srcPATH = '/content/training-a'
trainPATH = '/content/train'
testPATH = '/content/test'

os.mkdir(trainPATH)
os.mkdir(testPATH)

i = 0
src_files = os.listdir(srcPATH)
for file_name in data['filename']:
  full_file_name = os.path.join(srcPATH, file_name)
  if os.path.isfile(full_file_name):
    if i < split_val:
      shutil.copy(full_file_name, trainPATH)
    else:
      shutil.copy(full_file_name, testPATH)
    i += 1
print(i)

19702


In [ ]:
train_transform = transforms.Compose([
                                      transforms.RandomRotation(30),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor()
                                      ])

test_transform = transforms.Compose([
                                     transforms.ToTensor()
                                     ])

dataTrain = data.iloc[: split_val, :]
dataTest = data.iloc[split_val :, :]

train_data  = Dataset(dataTrain, trainPATH, train_transform)
test_data = Dataset(dataTest, testPATH, test_transform)
print(len(train_data))
print(len(test_data))

15761
3941


In [ ]:
dataTrain

,filename,digit
0,a18483.png,0
1,a07473.png,5
2,a18214.png,6
3,a06037.png,2
4,a12149.png,8
...,...,...
15756,a03161.png,0
15757,a08087.png,7
15758,a03861.png,3
15759,a12537.png,9


In [ ]:
dataTest

,filename,digit
15761,a09841.png,3
15762,a04007.png,0
15763,a12002.png,4
15764,a01497.png,8
15765,a11314.png,0
...,...,...
19697,a12025.png,5
19698,a18838.png,7
19699,a09484.png,2
19700,a13656.png,4


In [ ]:
# One Image Size
print(train_data[0][0].size())
print(train_data[0][0].numpy().shape)

torch.Size([1, 180, 180])
(1, 180, 180)


In [ ]:
# prepare loaders
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size = batch_size)

test_loader = torch.utils.data.DataLoader(
    test_data, 
    batch_size = batch_size)

print("Train dataloader:{}".format(len(train_loader)))
print("Test dataloader:{}".format(len(test_loader)))

Train dataloader:62
Test dataloader:16


In [ ]:
classes = data['digit'].unique().tolist()
classes.sort()
print(classes)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


## Define Model
---
Define and develop the neural network model.

In [ ]:
class NeuralNetworkModel(nn.Module):
  def __init__(self, input_size, num_classes, num_hidden, num_nodes):
    super().__init__()
    # Input layer
    self.input = nn.Linear(input_size, num_nodes)
    
    # All Activation Except Final
    self.activate_All = nn.ReLU()
    
    # Internal hidden layers
    self.hidden = nn.Linear(num_nodes, num_nodes)

    # Output hidden activation
    # self.activate_last = nn.Softmax(dim = 1)
    self.activate_last = nn.ReLU()

    # Output Layer
    self.out = nn.Linear(num_nodes, num_classes)

    # Storing Value of Hidden Layer Count
    self.hiddenCnt = num_hidden
    
  def forward(self, x):
    # Input Layer
    x = self.input(x)

    # 3 Hidden Layers
    for i in range(self.hiddenCnt):
      x = self.activate_All(x)
      x = self.hidden(x)
    
    # Output Layer
    x = self.activate_last(x)
    x = self.out(x)
    
    return x

In [ ]:
model = NeuralNetworkModel(input_size = input_dim,
                           num_classes = len(classes),
                           num_hidden = no_hidden_layer,
                           num_nodes = no_nodes)
# To enable GPU
model.to(device)

NeuralNetworkModel(
  (input): Linear(in_features=32400, out_features=200, bias=True)
  (activate_All): ReLU()
  (hidden): Linear(in_features=200, out_features=200, bias=True)
  (activate_last): ReLU()
  (out): Linear(in_features=200, out_features=10, bias=True)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
iter = 0
while True:
  for i, (images, labels) in enumerate(train_loader):
    images = images.view(-1, input_dim).to(device)
    labels = labels.to(device)

    # Clear gradients w.r.t. parameters
    optimizer.zero_grad()

    # Forward pass to get output/logits
    outputs = model(images) 

    # Calculate Loss: softmax --> cross entropy loss
    loss = criterion(outputs, labels)

    # Getting gradients w.r.t. parameters
    loss.backward()

    # Updating parameters
    optimizer.step()

    iter += 1

    if iter % 1000 == 0:
      # Calculate Accuracy         
      correctTest = 0
      correctTrain = 0
      totalTest = 0
      totalTrain = 0

      # Test Dataset
      for images, labels in test_loader:
        images = images.view(-1, input_dim).to(device)

        # Forward pass only to get logits/output
        outputs = model(images)

        # Get predictions from the maximum value
        _, predicted = torch.max(outputs, 1)

        # Total number of labels
        totalTest += labels.size(0)

        # Train dataset
        if torch.cuda.is_available():
          correctTest += (predicted.cpu() == labels.cpu()).sum() 
        else:
          correctTest += (predicted == labels).sum()
      
      # Train Dataset
      for images, labels in train_loader:
        images = images.view(-1, input_dim).to(device)

        # Forward pass only to get logits/output
        outputs = model(images)

        # Get predictions from the maximum value
        _, predicted = torch.max(outputs, 1)

        # Total number of labels
        totalTrain += labels.size(0)


        # Total correct predictions
        if torch.cuda.is_available():
          correctTrain += (predicted.cpu() == labels.cpu()).sum() 
        else:
          correctTrain += (predicted == labels).sum()

      accuracyTrain = 100 * correctTrain.item() / totalTrain
      accuracyTest = 100 * correctTest.item() / totalTest

      # Print Loss
      print('Iteration: {}. \t Loss: {}. \t Train Accuracy: {}. \t Test Accuracy: {}.'.format(iter, loss.item(), accuracyTrain, accuracyTest))
      
    if iter == iterations:
      break
  if iter == iterations:
    break

Iteration: 1000. 	 Loss: 2.301421880722046. 	 Train Accuracy: 12.568999428970242. 	 Test Accuracy: 9.743719868053793.
Iteration: 2000. 	 Loss: 2.2971320152282715. 	 Train Accuracy: 11.18583846202652. 	 Test Accuracy: 10.37807663029688.
Iteration: 3000. 	 Loss: 2.2941551208496094. 	 Train Accuracy: 14.586637903686315. 	 Test Accuracy: 15.300685105303222.
Iteration: 4000. 	 Loss: 2.2583675384521484. 	 Train Accuracy: 12.638791954825201. 	 Test Accuracy: 14.108094392286223.
Iteration: 5000. 	 Loss: 2.243220567703247. 	 Train Accuracy: 16.99765243322124. 	 Test Accuracy: 17.6604922608475.
Iteration: 6000. 	 Loss: 2.2192904949188232. 	 Train Accuracy: 20.36672799949242. 	 Test Accuracy: 20.274042121289014.
Iteration: 7000. 	 Loss: 2.202519178390503. 	 Train Accuracy: 21.553200939026713. 	 Test Accuracy: 20.93377315402182.
Iteration: 8000. 	 Loss: 2.109081745147705. 	 Train Accuracy: 19.326184886745764. 	 Test Accuracy: 18.853082973864502.
Iteration: 9000. 	 Loss: 2.0742621421813965. 	 Train

In [ ]:
torch.save(model.state_dict(), '/content/Assignment2_NonNormalized.pkl')